# LangChain + DatetimeOutputParser

In [1]:
from dotenv import load_dotenv
import os
# .env 파일을 불러와서 환경 변수로 설정
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:6])

gsk_yy


In [3]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import DatetimeOutputParser
from datetime import datetime

print(datetime.today())

# 출력 파서 초기화 (시간대 포함 가능)
datetime_parser = DatetimeOutputParser()
format_instructions = datetime_parser.get_format_instructions()

print("날짜 출력 형식 지시사항:")
print(format_instructions)

2025-09-30 09:24:53.130209
날짜 출력 형식 지시사항:
Write a datetime string that matches the following pattern: '%Y-%m-%dT%H:%M:%S.%fZ'.

Examples: 2023-07-04T14:30:00.000000Z, 1999-12-31T23:59:59.999999Z, 2025-01-01T00:00:00.000000Z

Return ONLY this string, no other words!


In [5]:

# 프롬프트 템플릿
# 현재 날짜를 명시적으로 프롬프트에 주입
current_date = datetime.now().strftime("%Y-%m-%d")
template = f"""
현재 날짜: {current_date}
다음 텍스트에서 날짜/시간 정보를 추출하세요. 상대적 표현(예: '다음 주 금요일')은 현재 날짜를 기준으로 계산합니다.
텍스트: {{text}}

{{format_instructions}}
"""

prompt = ChatPromptTemplate.from_template(template)
prompt = prompt.partial(format_instructions=format_instructions)
print(prompt)

model = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    #model="meta-llama/llama-4-scout-17b-16e-instruct",  # Spring AI와 동일한 모델
    model="moonshotai/kimi-k2-instruct-0905",
    temperature=0.1
)
current_date = datetime.now().strftime("%Y-%m-%d (%A)")
# 테스트 데이터 (다양한 날짜 형식 포함)
texts = [
    "회의는 한달 후 오후 2시에 예정되어 있습니다.",
    f"현재 날짜는 {current_date}입니다. 프로젝트 마감일은 다음 주 목요일입니다.",
    "행사 시작은 다음 달 첫날입니다.",
    "3일 후 오후 3시30분에 시스템 점검이 진행됩니다.",
    "지난주 금요일 정오에 전시회가 있었습니다."
]

# 체인 실행 및 결과 출력
chain = prompt | model | datetime_parser

for text in texts:
    print(f"\n원본 텍스트: {text}")
    output = chain.invoke({"text": text})
    print(f"추출된 날짜: {output.strftime('%Y-%m-%d %H:%M:%S %Z')}")

input_variables=['text'] input_types={} partial_variables={'format_instructions': "Write a datetime string that matches the following pattern: '%Y-%m-%dT%H:%M:%S.%fZ'.\n\nExamples: 2023-07-04T14:30:00.000000Z, 1999-12-31T23:59:59.999999Z, 2025-01-01T00:00:00.000000Z\n\nReturn ONLY this string, no other words!"} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['format_instructions', 'text'], input_types={}, partial_variables={}, template="\n현재 날짜: 2025-09-30\n다음 텍스트에서 날짜/시간 정보를 추출하세요. 상대적 표현(예: '다음 주 금요일')은 현재 날짜를 기준으로 계산합니다.\n텍스트: {text}\n\n{format_instructions}\n"), additional_kwargs={})]

원본 텍스트: 회의는 한달 후 오후 2시에 예정되어 있습니다.
추출된 날짜: 2025-10-30 14:00:00 

원본 텍스트: 현재 날짜는 2025-09-30 (Tuesday)입니다. 프로젝트 마감일은 다음 주 목요일입니다.
추출된 날짜: 2025-10-09 23:59:59 

원본 텍스트: 행사 시작은 다음 달 첫날입니다.
추출된 날짜: 2025-10-01 00:00:00 

원본 텍스트: 3일 후 오후 3시30분에 시스템 점검이 진행됩니다.
추출된 날짜: 2025-10-03 15:30:00 

원본 텍스트: 지난주 금요일 정오에 전시회가 있었습니다.
추출된 날짜: 2025-09-26 12:00:00 
